# Red Wine Quality Project
***
***

# Goal
***
- Predict the quality of red wines based on some of their various physiochemical attributes

# Setup
***

In [8]:
# establishing environment
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

# Acquire
Acquiring the data for this project
***

In [9]:
wines = pd.read_csv('winequality-red.csv')

wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


# Prepare
Preparing the data for this project

## Splitting data

In [10]:
# splitting data
train_validate, test = train_test_split(wines, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

## Scaling data

In [11]:
# identifying columns to scale
col_to_scale = []

for col in wines:
    if (wines[col].min() < 0 or wines[col].max() > 1) and col != 'quality':
        col_to_scale.append(col)

col_to_scale

['fixed acidity',
 'volatile acidity',
 'residual sugar',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

['fixed acidity',
 'volatile acidity',
 'residual sugar',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [53]:
# creating cols that will hold scaled values
for df in wine_d:
    for col in col_to_scale:
        wine_d[df][col + '_s'] = 0

In [57]:
# creating scaler object
scaler = sklearn.preprocessing.MinMaxScaler()

for col in col_to_scale:
    for train_col in train:
        if (train_col[-2:] == '_s') & (train_col[:-2] == col):
            train[train_col] = scaler.fit_transform(train[col])

ValueError: Expected 2D array, got 1D array instead:
array=[ 8.3  6.7  8.2  8.2 10.8  9.   7.2  8.6  7.   6.7  7.2 10.9  6.5  6.3
  7.9  6.2  8.5  7.2  7.2  8.2  8.5 10.2  7.8  7.8  9.9  8.   7.6  9.1
  7.5  9.9  5.6  6.6 10.3  7.8  8.2  7.2 12.2  7.1 12.   8.3  7.2  7.
  6.5  7.7  9.9 12.4  5.1  5.2  8.   7.9  7.2  7.9  7.6  7.7  7.  10.4
  7.6  7.5  8.4  7.9  6.1  9.4  7.  10.5  8.3 10.4  9.2  7.4  8.   8.5
  8.1  8.2  8.9  8.3  7.1  9.   7.8  8.6  6.8  9.1  6.4  6.2 12.5  9.9
  8.9 11.6  8.6  9.   7.4 11.6  7.1  7.5  7.4  5.4  9.4 12.   7.1 10.
  7.1  7.2  7.3  7.8 10.5 10.7  9.9  6.4 11.3  8.1  8.6  7.3 11.9  6.5
  7.4  9.1  8.6  7.8  8.4  7.   7.2 11.8  6.3  7.7 11.5  8.5  7.1  7.8
  7.6  6.2  8.2  6.   7.4  6.3 12.  10.   7.7  4.7  7.2 10.9 10.4  7.4
  8.   9.9  7.1 10.6 10.2  7.9  8.9  6.5 10.2  7.5  8.   6.8  7.8  9.7
  8.4  6.8 10.4  8.1  5.7  8.2  7.3  7.7  5.9 10.1  9.9  8.5  8.3  7.
  8.3  7.1  6.7  6.2 12.6  6.8  8.   6.9  7.1  9.3  7.8 10.8 10.1  9.3
  6.8  6.7  9.6  8.4 

ValueError: Expected 2D array, got 1D array instead:
array=[ 8.3  6.7  8.2  8.2 10.8  9.   7.2  8.6  7.   6.7  7.2 10.9  6.5  6.3
  7.9  6.2  8.5  7.2  7.2  8.2  8.5 10.2  7.8  7.8  9.9  8.   7.6  9.1
  7.5  9.9  5.6  6.6 10.3  7.8  8.2  7.2 12.2  7.1 12.   8.3  7.2  7.
  6.5  7.7  9.9 12.4  5.1  5.2  8.   7.9  7.2  7.9  7.6  7.7  7.  10.4
  7.6  7.5  8.4  7.9  6.1  9.4  7.  10.5  8.3 10.4  9.2  7.4  8.   8.5
  8.1  8.2  8.9  8.3  7.1  9.   7.8  8.6  6.8  9.1  6.4  6.2 12.5  9.9
  8.9 11.6  8.6  9.   7.4 11.6  7.1  7.5  7.4  5.4  9.4 12.   7.1 10.
  7.1  7.2  7.3  7.8 10.5 10.7  9.9  6.4 11.3  8.1  8.6  7.3 11.9  6.5
  7.4  9.1  8.6  7.8  8.4  7.   7.2 11.8  6.3  7.7 11.5  8.5  7.1  7.8
  7.6  6.2  8.2  6.   7.4  6.3 12.  10.   7.7  4.7  7.2 10.9 10.4  7.4
  8.   9.9  7.1 10.6 10.2  7.9  8.9  6.5 10.2  7.5  8.   6.8  7.8  9.7
  8.4  6.8 10.4  8.1  5.7  8.2  7.3  7.7  5.9 10.1  9.9  8.5  8.3  7.
  8.3  7.1  6.7  6.2 12.6  6.8  8.   6.9  7.1  9.3  7.8 10.8 10.1  9.3
  6.8  6.7  9.6  8.4 

In [ ]:
# fitting scaler to train column and scaling after
train[col_scaled_names] = scaler.fit_transform(train[col_to_scale])

# scaling data in validate and test dataframes
validate[col_scaled_names] = scaler.transform(validate[col_to_scale])
test[col_scaled_names] = scaler.transform(test[col_to_scale])

In [12]:
# creating scaler object
scaler = sklearn.preprocessing.MinMaxScaler()

# creating dict of dfs to iterate upon
# useful for being able to check df names during loops
wine_d = {'train' : train, 'validate': validate, 'test': test}

# creating cols that will hold scaled values
for df in wine_d:
    for col in col_to_scale:
        wine_d[df][col + '_s'] = 0

In [ ]:
# fitting scaler to train column and scaling after
train[col_scaled_names] = scaler.fit_transform(train[col_to_scale])

# scaling data in validate and test dataframes
validate[col_scaled_names] = scaler.transform(validate[col_to_scale])
test[col_scaled_names] = scaler.transform(test[col_to_scale])

In [52]:
for df in wine_d:
    if df == 'train':
        for col in wine_d[df]:
            if col[-2:] == '_s':
                wine_d[df][col] = scaler.fit_transform(wine_d[df][col[:-2]])
    #if str(df) != 'train':
     #   for col in df:
      #      if col[-2:] == '_s':
       #         df[col] = scaler.transform(df[col])

ValueError: Expected 2D array, got 1D array instead:
array=[ 8.3  6.7  8.2  8.2 10.8  9.   7.2  8.6  7.   6.7  7.2 10.9  6.5  6.3
  7.9  6.2  8.5  7.2  7.2  8.2  8.5 10.2  7.8  7.8  9.9  8.   7.6  9.1
  7.5  9.9  5.6  6.6 10.3  7.8  8.2  7.2 12.2  7.1 12.   8.3  7.2  7.
  6.5  7.7  9.9 12.4  5.1  5.2  8.   7.9  7.2  7.9  7.6  7.7  7.  10.4
  7.6  7.5  8.4  7.9  6.1  9.4  7.  10.5  8.3 10.4  9.2  7.4  8.   8.5
  8.1  8.2  8.9  8.3  7.1  9.   7.8  8.6  6.8  9.1  6.4  6.2 12.5  9.9
  8.9 11.6  8.6  9.   7.4 11.6  7.1  7.5  7.4  5.4  9.4 12.   7.1 10.
  7.1  7.2  7.3  7.8 10.5 10.7  9.9  6.4 11.3  8.1  8.6  7.3 11.9  6.5
  7.4  9.1  8.6  7.8  8.4  7.   7.2 11.8  6.3  7.7 11.5  8.5  7.1  7.8
  7.6  6.2  8.2  6.   7.4  6.3 12.  10.   7.7  4.7  7.2 10.9 10.4  7.4
  8.   9.9  7.1 10.6 10.2  7.9  8.9  6.5 10.2  7.5  8.   6.8  7.8  9.7
  8.4  6.8 10.4  8.1  5.7  8.2  7.3  7.7  5.9 10.1  9.9  8.5  8.3  7.
  8.3  7.1  6.7  6.2 12.6  6.8  8.   6.9  7.1  9.3  7.8 10.8 10.1  9.3
  6.8  6.7  9.6  8.4 

ValueError: Expected 2D array, got 1D array instead:
array=[ 8.3  6.7  8.2  8.2 10.8  9.   7.2  8.6  7.   6.7  7.2 10.9  6.5  6.3
  7.9  6.2  8.5  7.2  7.2  8.2  8.5 10.2  7.8  7.8  9.9  8.   7.6  9.1
  7.5  9.9  5.6  6.6 10.3  7.8  8.2  7.2 12.2  7.1 12.   8.3  7.2  7.
  6.5  7.7  9.9 12.4  5.1  5.2  8.   7.9  7.2  7.9  7.6  7.7  7.  10.4
  7.6  7.5  8.4  7.9  6.1  9.4  7.  10.5  8.3 10.4  9.2  7.4  8.   8.5
  8.1  8.2  8.9  8.3  7.1  9.   7.8  8.6  6.8  9.1  6.4  6.2 12.5  9.9
  8.9 11.6  8.6  9.   7.4 11.6  7.1  7.5  7.4  5.4  9.4 12.   7.1 10.
  7.1  7.2  7.3  7.8 10.5 10.7  9.9  6.4 11.3  8.1  8.6  7.3 11.9  6.5
  7.4  9.1  8.6  7.8  8.4  7.   7.2 11.8  6.3  7.7 11.5  8.5  7.1  7.8
  7.6  6.2  8.2  6.   7.4  6.3 12.  10.   7.7  4.7  7.2 10.9 10.4  7.4
  8.   9.9  7.1 10.6 10.2  7.9  8.9  6.5 10.2  7.5  8.   6.8  7.8  9.7
  8.4  6.8 10.4  8.1  5.7  8.2  7.3  7.7  5.9 10.1  9.9  8.5  8.3  7.
  8.3  7.1  6.7  6.2 12.6  6.8  8.   6.9  7.1  9.3  7.8 10.8 10.1  9.3
  6.8  6.7  9.6  8.4 

In [51]:
for df in wine_d:
    if df == 'train':
        for col in wine_d[df]:
            if col[-2:] == '_s':
                print(col[:-2])

fixed acidity
volatile acidity
residual sugar
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol
fixed acidity
volatile acidity
residual sugar
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol


In [28]:
wine_d['train']['density_s']

965     1
822     1
1481    1
247     1
250     1
       ..
862     1
998     1
1057    1
1345    1
635     1
Name: density_s, Length: 895, dtype: int64

965     1
822     1
1481    1
247     1
250     1
       ..
862     1
998     1
1057    1
1345    1
635     1
Name: density_s, Length: 895, dtype: int64